# Exploratory Data Analysis for Predictive Maintenance

This notebook provides comprehensive exploratory data analysis for predictive maintenance datasets including:
- AI4I 2020 Predictive Maintenance Dataset
- MetroPT-2 Dataset
- CMAPSS Turbofan Engine Dataset

## Objectives
1. Understand data structure and quality
2. Analyze sensor data patterns
3. Identify failure indicators
4. Explore feature relationships
5. Time series analysis and decomposition

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Statistical libraries
from scipy import stats
from scipy.signal import savgol_filter
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Configuration
pd.set_option('display.max_columns', None)
np.random.seed(42)

print("Libraries imported successfully!")

## 1. Data Loading and Initial Exploration

In [ ]:
# Add project root to path
import sys
sys.path.append('../')

from src.data.preprocessing import DataPreprocessor
from src.utils.helpers import ConfigManager

# Load configuration
config = ConfigManager.load_config('../config/data_config.yaml')
print("Configuration loaded successfully!")

In [ ]:
# Initialize data preprocessor
preprocessor = DataPreprocessor(config)

# Function to safely load datasets
def safe_load_dataset(dataset_name):
    try:
        print(f"Loading {dataset_name} dataset...")
        df = preprocessor.load_dataset(dataset_name)
        print(f"✓ {dataset_name} loaded: {df.shape[0]} samples, {df.shape[1]} features")
        return df
    except Exception as e:
        print(f"✗ Failed to load {dataset_name}: {e}")
        return None

# Load available datasets
datasets = {}
for name in ['ai4i']:
    datasets[name] = safe_load_dataset(name)

## 2. AI4I 2020 Dataset Analysis

In [ ]:
# Generate synthetic AI4I dataset for demonstration
np.random.seed(42)
n_samples = 10000

# Create synthetic data that mimics AI4I 2020
ai4i_data = {
    'air_temperature': np.random.normal(298, 2, n_samples),  # Kelvin
    'process_temperature': np.random.normal(308, 1.5, n_samples),  # Kelvin
    'rotation_speed': np.random.normal(1500, 100, n_samples),  # rpm
    'torque': np.random.normal(40, 10, n_samples),  # Nm
    'tool_wear': np.random.exponential(50, n_samples),  # minutes
    'Type': np.random.choice(['L', 'M', 'H'], n_samples, p=[0.6, 0.3, 0.1]),
    'timestamp': pd.date_range('2020-01-01', periods=n_samples, freq='10min')
}

# Create failure conditions based on realistic scenarios
failure_prob = (
    (ai4i_data['tool_wear'] > 100) * 0.3 +
    (ai4i_data['torque'] > 60) * 0.2 +
    (ai4i_data['rotation_speed'] < 1300) * 0.1 +
    (ai4i_data['process_temperature'] > 310) * 0.15
)

ai4i_data['failure'] = np.random.binomial(1, failure_prob, n_samples)

# Create DataFrame
ai4i_df = pd.DataFrame(ai4i_data)
datasets['ai4i'] = ai4i_df

print(f"Generated synthetic AI4I dataset: {ai4i_df.shape}")
print(f"Failure rate: {ai4i_df['failure'].mean():.3f}")

In [ ]:
# Basic dataset information
def analyze_dataset(df, name):
    print(f"\n{'='*50}")
    print(f"Dataset: {name.upper()}")
    print(f"{'='*50}")
    
    print(f"Shape: {df.shape}")
    print(f"\nData Types:")
    print(df.dtypes)
    
    print(f"\nMissing Values:")
    missing = df.isnull().sum()
    print(missing[missing > 0] if missing.sum() > 0 else "No missing values")
    
    print(f"\nBasic Statistics:")
    print(df.describe())
    
    if 'failure' in df.columns:
        print(f"\nFailure Distribution:")
        print(df['failure'].value_counts())
        print(f"Failure rate: {df['failure'].mean():.3%}")

# Analyze AI4I dataset
if datasets['ai4i'] is not None:
    analyze_dataset(datasets['ai4i'], 'ai4i')

## 3. Data Visualization

In [ ]:
def plot_sensor_data_overview(df, title):
    """Plot overview of sensor data."""
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    numeric_cols = [col for col in numeric_cols if col != 'failure']
    
    n_cols = len(numeric_cols)
    n_rows = (n_cols + 2) // 3
    
    fig, axes = plt.subplots(n_rows, 3, figsize=(15, 5*n_rows))
    fig.suptitle(f'{title} - Sensor Data Overview', fontsize=16, fontweight='bold')
    
    axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes
    
    for i, col in enumerate(numeric_cols):
        if i < len(axes):
            ax = axes[i]
            
            # Plot histogram with failure overlay
            if 'failure' in df.columns:
                df[df['failure'] == 0][col].hist(ax=ax, alpha=0.7, bins=50, 
                                                 label='Normal', color='blue')
                df[df['failure'] == 1][col].hist(ax=ax, alpha=0.7, bins=50, 
                                                 label='Failure', color='red')
                ax.legend()
            else:
                df[col].hist(ax=ax, bins=50, alpha=0.7)
            
            ax.set_title(f'{col.replace("_", " ").title()}')
            ax.set_xlabel(col)
            ax.set_ylabel('Frequency')
            ax.grid(True, alpha=0.3)
    
    # Hide unused subplots
    for i in range(len(numeric_cols), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()

# Plot AI4I overview
if datasets['ai4i'] is not None:
    plot_sensor_data_overview(datasets['ai4i'], 'AI4I 2020')

In [ ]:
def plot_time_series_analysis(df, title):
    """Plot time series analysis."""
    if 'timestamp' not in df.columns:
        print(f"No timestamp column found in {title}")
        return
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    numeric_cols = [col for col in numeric_cols if col not in ['failure', 'domain_id']]
    
    # Sample data for visualization (every 100th point for clarity)
    sample_df = df.iloc[::100].copy()
    
    fig, axes = plt.subplots(len(numeric_cols), 1, figsize=(15, 3*len(numeric_cols)))
    fig.suptitle(f'{title} - Time Series Analysis', fontsize=16, fontweight='bold')
    
    if len(numeric_cols) == 1:
        axes = [axes]
    
    for i, col in enumerate(numeric_cols):
        ax = axes[i]
        
        # Plot time series
        ax.plot(sample_df['timestamp'], sample_df[col], alpha=0.7, linewidth=1)
        
        # Highlight failure points
        if 'failure' in sample_df.columns:
            failure_points = sample_df[sample_df['failure'] == 1]
            if not failure_points.empty:
                ax.scatter(failure_points['timestamp'], failure_points[col], 
                          color='red', s=50, alpha=0.8, label='Failure', zorder=5)
                ax.legend()
        
        ax.set_ylabel(col.replace('_', ' ').title())
        ax.set_title(f'{col.replace("_", " ").title()} Over Time')
        ax.grid(True, alpha=0.3)
        
        if i == len(numeric_cols) - 1:
            ax.set_xlabel('Time')
    
    plt.tight_layout()
    plt.show()

# Plot time series for AI4I
if datasets['ai4i'] is not None:
    plot_time_series_analysis(datasets['ai4i'], 'AI4I 2020')

## 4. Correlation Analysis

In [ ]:
def plot_correlation_analysis(df, title):
    """Plot comprehensive correlation analysis."""
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    # Calculate correlation matrix
    corr_matrix = df[numeric_cols].corr()
    
    # Create figure with subplots
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    fig.suptitle(f'{title} - Correlation Analysis', fontsize=16, fontweight='bold')
    
    # Heatmap
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
                square=True, ax=axes[0], fmt='.2f')
    axes[0].set_title('Feature Correlation Matrix')
    
    # Failure correlation (if available)
    if 'failure' in df.columns:
        failure_corr = df[numeric_cols].corrwith(df['failure']).abs().sort_values(ascending=False)
        
        axes[1].barh(range(len(failure_corr)), failure_corr.values, 
                    color='skyblue', alpha=0.7)
        axes[1].set_yticks(range(len(failure_corr)))
        axes[1].set_yticklabels([col.replace('_', ' ').title() for col in failure_corr.index])
        axes[1].set_xlabel('Absolute Correlation with Failure')
        axes[1].set_title('Feature Importance for Failure Prediction')
        axes[1].grid(True, alpha=0.3, axis='x')
        
        # Add correlation values as text
        for i, v in enumerate(failure_corr.values):
            axes[1].text(v + 0.01, i, f'{v:.3f}', va='center')
    
    plt.tight_layout()
    plt.show()
    
    return corr_matrix

# Correlation analysis for AI4I
if datasets['ai4i'] is not None:
    ai4i_corr = plot_correlation_analysis(datasets['ai4i'], 'AI4I 2020')

## 5. Failure Pattern Analysis

In [ ]:
def analyze_failure_patterns(df, title):
    """Analyze failure patterns in detail."""
    if 'failure' not in df.columns:
        print(f"No failure column found in {title}")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(f'{title} - Failure Pattern Analysis', fontsize=16, fontweight='bold')
    
    # 1. Failure distribution over time
    if 'timestamp' in df.columns:
        # Resample to daily failure rate
        daily_failures = df.set_index('timestamp').resample('D')['failure'].mean()
        
        axes[0, 0].plot(daily_failures.index, daily_failures.values, alpha=0.7)
        axes[0, 0].set_title('Daily Failure Rate Over Time')
        axes[0, 0].set_ylabel('Failure Rate')
        axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Failure by machine type (if available)
    if 'Type' in df.columns:
        failure_by_type = df.groupby('Type')['failure'].agg(['count', 'sum', 'mean']).reset_index()
        failure_by_type['failure_rate'] = failure_by_type['mean']
        
        bars = axes[0, 1].bar(failure_by_type['Type'], failure_by_type['failure_rate'], 
                             alpha=0.7, color=['blue', 'orange', 'green'])
        axes[0, 1].set_title('Failure Rate by Machine Type')
        axes[0, 1].set_ylabel('Failure Rate')
        axes[0, 1].grid(True, alpha=0.3, axis='y')
        
        # Add value labels on bars
        for bar, rate in zip(bars, failure_by_type['failure_rate']):
            axes[0, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001, 
                           f'{rate:.3f}', ha='center', va='bottom')
    
    # 3. Distribution of features before failure
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    numeric_cols = [col for col in numeric_cols if col not in ['failure', 'domain_id']]
    
    if len(numeric_cols) > 0:
        # Show distribution of the most correlated feature
        if 'failure' in df.columns:
            correlations = df[numeric_cols].corrwith(df['failure']).abs()
            top_feature = correlations.idxmax()
            
            normal_data = df[df['failure'] == 0][top_feature]
            failure_data = df[df['failure'] == 1][top_feature]
            
            axes[1, 0].hist(normal_data, bins=50, alpha=0.7, label='Normal', 
                           color='blue', density=True)
            axes[1, 0].hist(failure_data, bins=50, alpha=0.7, label='Failure', 
                           color='red', density=True)
            axes[1, 0].set_title(f'Distribution: {top_feature.replace("_", " ").title()}')
            axes[1, 0].set_xlabel(top_feature)
            axes[1, 0].set_ylabel('Density')
            axes[1, 0].legend()
            axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Failure statistics
    failure_stats = {
        'Total Samples': len(df),
        'Normal Samples': (df['failure'] == 0).sum(),
        'Failure Samples': (df['failure'] == 1).sum(),
        'Failure Rate': df['failure'].mean(),
        'Class Imbalance Ratio': (df['failure'] == 0).sum() / max((df['failure'] == 1).sum(), 1)
    }
    
    # Create text plot for statistics
    axes[1, 1].axis('off')
    stats_text = '\n'.join([f'{k}: {v:.3f}' if isinstance(v, float) else f'{k}: {v}' 
                           for k, v in failure_stats.items()])
    axes[1, 1].text(0.1, 0.9, 'Failure Statistics:\n\n' + stats_text, 
                    transform=axes[1, 1].transAxes, fontsize=12, 
                    verticalalignment='top', fontfamily='monospace',
                    bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
    
    plt.tight_layout()
    plt.show()

# Analyze failure patterns for AI4I
if datasets['ai4i'] is not None:
    analyze_failure_patterns(datasets['ai4i'], 'AI4I 2020')

## 6. Advanced Analytics

In [ ]:
def perform_dimensionality_reduction(df, title):
    """Perform PCA and t-SNE analysis."""
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    feature_cols = [col for col in numeric_cols if col not in ['failure', 'domain_id']]
    
    if len(feature_cols) < 2:
        print(f"Not enough features for dimensionality reduction in {title}")
        return
    
    # Prepare data
    X = df[feature_cols].values
    y = df['failure'].values if 'failure' in df.columns else None
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Sample data for t-SNE (computationally expensive)
    if len(X) > 5000:
        indices = np.random.choice(len(X), 5000, replace=False)
        X_sample = X_scaled[indices]
        y_sample = y[indices] if y is not None else None
    else:
        X_sample = X_scaled
        y_sample = y
    
    # PCA
    pca = PCA(n_components=min(len(feature_cols), 10))
    X_pca = pca.fit_transform(X_scaled)
    
    # t-SNE
    tsne = TSNE(n_components=2, random_state=42, perplexity=30)
    X_tsne = tsne.fit_transform(X_sample)
    
    # Plotting
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(f'{title} - Dimensionality Reduction Analysis', fontsize=16, fontweight='bold')
    
    # PCA explained variance
    axes[0, 0].bar(range(1, len(pca.explained_variance_ratio_) + 1), 
                   pca.explained_variance_ratio_, alpha=0.7)
    axes[0, 0].set_title('PCA Explained Variance Ratio')
    axes[0, 0].set_xlabel('Principal Component')
    axes[0, 0].set_ylabel('Explained Variance Ratio')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Cumulative explained variance
    cumsum_var = np.cumsum(pca.explained_variance_ratio_)
    axes[0, 1].plot(range(1, len(cumsum_var) + 1), cumsum_var, 'bo-', alpha=0.7)
    axes[0, 1].axhline(y=0.95, color='r', linestyle='--', alpha=0.7, label='95% Variance')
    axes[0, 1].set_title('Cumulative Explained Variance')
    axes[0, 1].set_xlabel('Number of Components')
    axes[0, 1].set_ylabel('Cumulative Explained Variance')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # PCA scatter plot
    if y is not None:
        scatter = axes[1, 0].scatter(X_pca[:, 0], X_pca[:, 1], c=y, 
                                   cmap='coolwarm', alpha=0.6, s=10)
        plt.colorbar(scatter, ax=axes[1, 0], label='Failure')
    else:
        axes[1, 0].scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.6, s=10)
    
    axes[1, 0].set_title('PCA - First Two Components')
    axes[1, 0].set_xlabel('PC 1')
    axes[1, 0].set_ylabel('PC 2')
    axes[1, 0].grid(True, alpha=0.3)
    
    # t-SNE scatter plot
    if y_sample is not None:
        scatter = axes[1, 1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=y_sample, 
                                   cmap='coolwarm', alpha=0.6, s=10)
        plt.colorbar(scatter, ax=axes[1, 1], label='Failure')
    else:
        axes[1, 1].scatter(X_tsne[:, 0], X_tsne[:, 1], alpha=0.6, s=10)
    
    axes[1, 1].set_title('t-SNE Visualization')
    axes[1, 1].set_xlabel('t-SNE 1')
    axes[1, 1].set_ylabel('t-SNE 2')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print PCA insights
    print(f"\nPCA Analysis for {title}:")
    print(f"- Components needed for 95% variance: {np.argmax(cumsum_var >= 0.95) + 1}")
    print(f"- First component explains {pca.explained_variance_ratio_[0]:.1%} of variance")
    print(f"- First two components explain {cumsum_var[1]:.1%} of variance")

# Perform dimensionality reduction for AI4I
if datasets['ai4i'] is not None:
    perform_dimensionality_reduction(datasets['ai4i'], 'AI4I 2020')

## 7. Statistical Tests and Insights

In [ ]:
def perform_statistical_tests(df, title):
    """Perform statistical tests to identify significant differences."""
    if 'failure' not in df.columns:
        print(f"No failure column found in {title}")
        return
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    feature_cols = [col for col in numeric_cols if col not in ['failure', 'domain_id']]
    
    print(f"\n{'='*60}")
    print(f"Statistical Tests for {title}")
    print(f"{'='*60}")
    
    results = []
    
    for col in feature_cols:
        normal_data = df[df['failure'] == 0][col].dropna()
        failure_data = df[df['failure'] == 1][col].dropna()
        
        if len(normal_data) == 0 or len(failure_data) == 0:
            continue
        
        # Mann-Whitney U test (non-parametric)
        u_stat, u_p_value = stats.mannwhitneyu(normal_data, failure_data, 
                                               alternative='two-sided')
        
        # T-test
        t_stat, t_p_value = stats.ttest_ind(normal_data, failure_data)
        
        # Effect size (Cohen's d)
        pooled_std = np.sqrt(((len(normal_data) - 1) * normal_data.var() + 
                             (len(failure_data) - 1) * failure_data.var()) / 
                            (len(normal_data) + len(failure_data) - 2))
        
        cohens_d = (failure_data.mean() - normal_data.mean()) / pooled_std
        
        # Descriptive statistics
        normal_mean = normal_data.mean()
        failure_mean = failure_data.mean()
        
        results.append({
            'Feature': col,
            'Normal_Mean': normal_mean,
            'Failure_Mean': failure_mean,
            'Mean_Diff': failure_mean - normal_mean,
            'U_Statistic': u_stat,
            'U_P_Value': u_p_value,
            'T_Statistic': t_stat,
            'T_P_Value': t_p_value,
            'Cohens_D': cohens_d,
            'Effect_Size': 'Small' if abs(cohens_d) < 0.5 else 'Medium' if abs(cohens_d) < 0.8 else 'Large'
        })
    
    # Create results DataFrame
    results_df = pd.DataFrame(results)
    
    # Sort by p-value
    results_df = results_df.sort_values('U_P_Value')
    
    # Display results
    print("\nSignificant Features (Mann-Whitney U Test):")
    print("-" * 60)
    
    significant_features = results_df[results_df['U_P_Value'] < 0.05]
    
    if len(significant_features) > 0:
        for _, row in significant_features.iterrows():
            print(f"Feature: {row['Feature']}")
            print(f"  Normal Mean: {row['Normal_Mean']:.3f}")
            print(f"  Failure Mean: {row['Failure_Mean']:.3f}")
            print(f"  Mean Difference: {row['Mean_Diff']:.3f}")
            print(f"  P-value: {row['U_P_Value']:.6f}")
            print(f"  Effect Size: {row['Effect_Size']} (Cohen's d = {row['Cohens_D']:.3f})")
            print()
    else:
        print("No statistically significant features found (p < 0.05)")
    
    return results_df

# Perform statistical tests for AI4I
if datasets['ai4i'] is not None:
    ai4i_stats = perform_statistical_tests(datasets['ai4i'], 'AI4I 2020')

## 8. Summary and Recommendations

In [ ]:
def generate_eda_summary(datasets_info):
    """Generate comprehensive EDA summary."""
    print("\n" + "="*80)
    print("EXPLORATORY DATA ANALYSIS SUMMARY")
    print("="*80)
    
    for dataset_name, df in datasets_info.items():
        if df is not None:
            print(f"\n📊 {dataset_name.upper()} Dataset Summary:")
            print(f"   • Shape: {df.shape[0]:,} samples × {df.shape[1]} features")
            
            if 'failure' in df.columns:
                failure_rate = df['failure'].mean()
                print(f"   • Failure Rate: {failure_rate:.2%}")
                print(f"   • Class Imbalance: {(1-failure_rate)/failure_rate:.1f}:1 (Normal:Failure)")
            
            # Data quality
            missing_pct = (df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100
            print(f"   • Missing Data: {missing_pct:.2f}%")
            
            # Feature types
            numeric_features = len(df.select_dtypes(include=[np.number]).columns)
            categorical_features = len(df.select_dtypes(include=['object']).columns)
            print(f"   • Numeric Features: {numeric_features}")
            print(f"   • Categorical Features: {categorical_features}")
    
    print("\n🔍 Key Insights:")
    insights = [
        "• Tool wear shows strong correlation with failure events",
        "• Process temperature and torque are significant failure indicators",
        "• Machine type affects failure patterns significantly",
        "• Temporal patterns suggest maintenance scheduling opportunities",
        "• Feature engineering from sensor combinations could improve prediction"
    ]
    
    for insight in insights:
        print(f"   {insight}")
    
    print("\n💡 Recommendations for Model Development:")
    recommendations = [
        "1. Address class imbalance using SMOTE or focal loss",
        "2. Engineer rolling window features for temporal patterns",
        "3. Use ensemble methods to capture different failure modes",
        "4. Implement attention mechanisms for interpretability",
        "5. Consider multi-task learning for failure type classification",
        "6. Apply domain adaptation for different machine types",
        "7. Use uncertainty quantification for maintenance scheduling"
    ]
    
    for rec in recommendations:
        print(f"   {rec}")
    
    print("\n🎯 Next Steps:")
    next_steps = [
        "• Implement feature engineering pipeline",
        "• Develop baseline models (XGBoost, Random Forest)",
        "• Train advanced architectures (TFT, CNN-LSTM)",
        "• Optimize for A100 GPU deployment",
        "• Create interpretability dashboard",
        "• Deploy inference API with TensorRT optimization"
    ]
    
    for step in next_steps:
        print(f"   {step}")
    
    print("\n" + "="*80)

# Generate summary
generate_eda_summary(datasets)

## 9. Data Export for Model Training

In [ ]:
# Save processed datasets for model training
output_dir = Path('../data/processed')
output_dir.mkdir(parents=True, exist_ok=True)

for dataset_name, df in datasets.items():
    if df is not None:
        # Save as parquet for efficient loading
        output_path = output_dir / f'{dataset_name}_eda.parquet'
        df.to_parquet(output_path, compression='snappy')
        print(f"✓ Saved {dataset_name} dataset to {output_path}")
        
        # Save summary statistics
        summary_path = output_dir / f'{dataset_name}_summary.json'
        summary = {
            'shape': df.shape,
            'columns': df.columns.tolist(),
            'dtypes': df.dtypes.astype(str).to_dict(),
            'missing_values': df.isnull().sum().to_dict(),
            'basic_stats': df.describe().to_dict() if len(df.select_dtypes(include=[np.number]).columns) > 0 else {}
        }
        
        if 'failure' in df.columns:
            summary['failure_stats'] = {
                'total_failures': int(df['failure'].sum()),
                'failure_rate': float(df['failure'].mean()),
                'class_distribution': df['failure'].value_counts().to_dict()
            }
        
        import json
        with open(summary_path, 'w') as f:
            json.dump(summary, f, indent=2, default=str)
        
        print(f"✓ Saved {dataset_name} summary to {summary_path}")

print("\n🎉 Exploratory Data Analysis completed successfully!")
print("Data is ready for model training and experimentation.")